In [3]:
import os
import sqlite3
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


def download_excel(url: str, css_selector: str) -> None:
    """
    Downloads the excel data file from the given URL and saves it to the current directory.
    
    Parameters:
        url (str): The URL to the data file.
        css_selector (str): The CSS selector for the download button.

    Returns:    
        None
    """
    s = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s)
    driver.get(url)

    # Locate the download button and get the CSV file URL
    download_button = driver.find_element(By.CSS_SELECTOR, '.wp-block-button__link')
    csv_url = download_button.get_attribute('href')

    # Download the CSV file
    response = requests.get(csv_url)
    with open('skill_test_data.xlsx', 'wb') as f:
        f.write(response.content)

    # Close the webdriver
    driver.quit()


def create_pivot_table(data: pd.DataFrame) -> pd.DataFrame:
    """
    Creates a pivot table from the given data and saves it to a new Excel file.

    Parameters:
        data (DataFrame): The data to create the pivot table from.
    
    Returns:
        DataFrame: The pivot table.
    """
    pivot_table = pd.pivot_table(data, index='Platform (Northbeam)',
                                values=['Spend', 'Attributed Rev (1d)', 'Imprs',
                                        'Visits', 'New Visits', 'Transactions (1d)',
                                        'Email Signups (1d)'],
                                aggfunc='sum')

    column_sort = ['Spend', 'Attributed Rev (1d)', 'Imprs',
                                        'Visits', 'New Visits', 'Transactions (1d)',
                                        'Email Signups (1d)']

    # Sort the columns
    pivot_table = pivot_table.reindex(column_sort, axis=1)

    # Rename the column headers
    pivot_table.columns = ['Sum of Spend', 'Sum of Attributed Rev (1d)', 'Sum of Imprs',
                        'Sum of Visits', 'Sum of New Visits', 'Sum of Transactions (1d)',
                        'Sum of Email Signups (1d)']

    # Sort by revenue descending
    pivot_table = pivot_table.sort_values('Sum of Attributed Rev (1d)', ascending=False)

    for column in pivot_table.columns:
        pivot_table[column] = pivot_table[column].apply(lambda x: f"{x:,.2f}")

    # Apply the formatting for the "Sum of Spend" and "Sum of Attributed Rev (1d)" columns
    pivot_table['Sum of Spend'] = pivot_table['Sum of Spend'].apply(lambda x: f"${x}")
    pivot_table['Sum of Attributed Rev (1d)'] = pivot_table['Sum of Attributed Rev (1d)'].apply(lambda x: f"${x}")

    return pivot_table


def create_database_and_insert_data(database_name: str, table_name: str, data: pd.DataFrame) -> None:
    """
    Creates a new SQLite database and inserts the given data into a new table.

    Parameters:
        database_name (str): The name of the database.
        table_name (str): The name of the table.
        data (DataFrame): The data to insert into the table.

    Returns:
        None
    """
    conn = sqlite3.connect(database_name)
    data.to_sql(table_name, conn, if_exists='replace')
    conn.commit()
    conn.close()


if __name__ == "__main__":
    # Step 1: Download the data file
    url = "https://jobs.homesteadstudio.co/data-engineer/assessment/download/"
    css_selector = ".wp-block-button__link'"
    download_excel(url, css_selector)

    # Step 2: Create a pivot table
    input_file = "skill_test_data.xlsx"
    data = pd.read_excel(input_file, sheet_name="data")
    pivot_table = create_pivot_table(data)

    # Save the pivot table to a new Excel file
    pivot_table.to_excel("pivot_table.xlsx")

    # Step 3 and 4: Create a new SQLite database and insert the pivot table
    database_name = "african_cow.db"
    table_name = "pivot_table"
    create_database_and_insert_data(database_name, table_name, pivot_table)

    print("Process completed.")

        Date Platform (Northbeam)                            Name  Status   
0 2022-11-26         Facebook Ads  02_Prospecting_$LC_Testing_ABO  Active  \
1 2022-11-27         Facebook Ads  02_Prospecting_$LC_Testing_ABO  Active   
2 2022-11-28         Facebook Ads  02_Prospecting_$LC_Testing_ABO  Active   
3 2022-11-29         Facebook Ads  02_Prospecting_$LC_Testing_ABO  Active   
4 2022-11-30         Facebook Ads  02_Prospecting_$LC_Testing_ABO  Active   

  Medium Source       Spend  Attributed Rev (1d)  LTV Attributed Rev   
0    NaN    NaN  517.530000             342.4250            342.4250  \
1    NaN    NaN  555.820000             207.2200            345.7000   
2    NaN    NaN  540.560000             263.9700            263.9700   
3    NaN    NaN  523.920001              81.9100             81.9100   
4    NaN    NaN  509.619999             229.6225            285.6225   

   Forecasted Attributed Rev (30d)  ...  LTV Transactions   
0                       342.904505  ...    

In [4]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('african_cow.db')

# Read the data from the "pivot_table" and store it in a DataFrame
data_from_db = pd.read_sql_query('SELECT * FROM pivot_table', conn)

# Print the data
print(data_from_db)

# Close the connection
conn.close()

   Platform (Northbeam) Sum of Spend Sum of Attributed Rev (1d)  Sum of Imprs   
0            Google Ads   $28,228.10                 $90,448.41  1,476,413.00  \
1          Facebook Ads   $30,073.42                 $49,872.23  1,225,850.00   
2               Organic        $0.00                 $45,056.68          0.00   
3        Organic Search        $0.00                 $24,479.03          0.00   
4          Unattributed        $0.00                  $4,860.46          0.00   
5     Instagram Organic        $0.00                  $2,614.29          0.00   
6                 Other        $0.00                  $2,205.55          0.00   
7           Other Email        $0.00                  $1,521.82          0.00   
8                TikTok    $1,108.45                  $1,360.64     58,796.00   
9      Facebook Organic        $0.00                  $1,231.29          0.00   
10      YouTube Organic        $0.00                    $450.83          0.00   
11              SMSBump     